# convert flat data (a csv spreadsheet) to structured data (a fact table and 5 dimension tables)


In [1]:
import pandas as pd 
df = pd.read_csv("data/uber_data.csv")
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80


In [2]:
# Check if the data is in proper formats. 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               100000 non-null  int64  
 1   tpep_pickup_datetime   100000 non-null  object 
 2   tpep_dropoff_datetime  100000 non-null  object 
 3   passenger_count        100000 non-null  int64  
 4   trip_distance          100000 non-null  float64
 5   pickup_longitude       100000 non-null  float64
 6   pickup_latitude        100000 non-null  float64
 7   RatecodeID             100000 non-null  int64  
 8   store_and_fwd_flag     100000 non-null  object 
 9   dropoff_longitude      100000 non-null  float64
 10  dropoff_latitude       100000 non-null  float64
 11  payment_type           100000 non-null  int64  
 12  fare_amount            100000 non-null  float64
 13  extra                  100000 non-null  float64
 14  mta_tax                100000 non-nul

In [3]:
# The datetime is not in proper format, 
#   so we manually convert them into datetime type. 
# We need to make sure it is actual datetime type so that we can use some 
#  datetime functions to extract the month, week etc. 

# Fix data types:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               100000 non-null  int64         
 1   tpep_pickup_datetime   100000 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  100000 non-null  datetime64[ns]
 3   passenger_count        100000 non-null  int64         
 4   trip_distance          100000 non-null  float64       
 5   pickup_longitude       100000 non-null  float64       
 6   pickup_latitude        100000 non-null  float64       
 7   RatecodeID             100000 non-null  int64         
 8   store_and_fwd_flag     100000 non-null  object        
 9   dropoff_longitude      100000 non-null  float64       
 10  dropoff_latitude       100000 non-null  float64       
 11  payment_type           100000 non-null  int64         
 12  fare_amount            100000 non-null  float

In [4]:
# Drop duplicate records and reset index
df = df.drop_duplicates().reset_index(drop=True) 
# make the index column explicit 
df['trip_id'] = df.index  
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               100000 non-null  int64         
 1   tpep_pickup_datetime   100000 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  100000 non-null  datetime64[ns]
 3   passenger_count        100000 non-null  int64         
 4   trip_distance          100000 non-null  float64       
 5   pickup_longitude       100000 non-null  float64       
 6   pickup_latitude        100000 non-null  float64       
 7   RatecodeID             100000 non-null  int64         
 8   store_and_fwd_flag     100000 non-null  object        
 9   dropoff_longitude      100000 non-null  float64       
 10  dropoff_latitude       100000 non-null  float64       
 11  payment_type           100000 non-null  int64         
 12  fare_amount            100000 non-null  float

In [15]:
# create first dimension table: datetime_dim 
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].drop_duplicates().reset_index(drop=True) 

datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday
 
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

# set the primary key 
datetime_dim['datetime_id'] = datetime_dim.index

# Reorder the columns so that PK is the first column 
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']] 
print(datetime_dim.shape) 
datetime_dim.info()



(99853, 13)
0       2016-03-01 00:00:00
1       2016-03-01 00:00:00
2       2016-03-01 00:00:00
3       2016-03-01 00:00:00
4       2016-03-01 00:00:01
                ...        
99848   2016-03-01 06:17:10
99849   2016-03-01 06:17:10
99850   2016-03-01 06:17:10
99851   2016-03-01 06:17:10
99852   2016-03-01 06:17:11
Name: tpep_pickup_datetime, Length: 99853, dtype: datetime64[ns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99853 entries, 0 to 99852
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   datetime_id            99853 non-null  int64         
 1   tpep_pickup_datetime   99853 non-null  datetime64[ns]
 2   pick_hour              99853 non-null  int64         
 3   pick_day               99853 non-null  int64         
 4   pick_month             99853 non-null  int64         
 5   pick_year              99853 non-null  int64         
 6   pick_weekday           99853 

14


In [6]:
# create a dimension table: passenger_count_dim 
passenger_count_dim = df[['passenger_count']].drop_duplicates().reset_index(drop=True)       
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index   # set the PK
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']] # make PK the first column

passenger_count_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   passenger_count_id  7 non-null      int64
 1   passenger_count     7 non-null      int64
dtypes: int64(2)
memory usage: 176.0 bytes


In [7]:
# create a dimension table: trip_distance_dim 
trip_distance_dim = df[['trip_distance']].drop_duplicates().reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]
trip_distance_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2303 entries, 0 to 2302
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   trip_distance_id  2303 non-null   int64  
 1   trip_distance     2303 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 36.0 KB


In [8]:
# create a dimension table: rate_code_dim 

# we want the actual meaning rather than the numeric codes for the categorical data.
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = df[['RatecodeID']].drop_duplicates().reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]
rate_code_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   rate_code_id    6 non-null      int64 
 1   RatecodeID      6 non-null      int64 
 2   rate_code_name  6 non-null      object
dtypes: int64(2), object(1)
memory usage: 184.0+ bytes


In [9]:
# create a dimension table: pickup_location_dim 
pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']].drop_duplicates().reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']]
pickup_location_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98055 entries, 0 to 98054
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   pickup_location_id  98055 non-null  int64  
 1   pickup_latitude     98055 non-null  float64
 2   pickup_longitude    98055 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 2.2 MB


In [10]:
# create a dimension table: dropoff_location_dim 
dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']].drop_duplicates().reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]
dropoff_location_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98417 entries, 0 to 98416
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   dropoff_location_id  98417 non-null  int64  
 1   dropoff_latitude     98417 non-null  float64
 2   dropoff_longitude    98417 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 2.3 MB


In [11]:
# create a dimension table: payment_type_dim 
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].drop_duplicates().reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name) 
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]
payment_type_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   payment_type_id    4 non-null      int64 
 1   payment_type       4 non-null      int64 
 2   payment_type_name  4 non-null      object
dtypes: int64(2), object(1)
memory usage: 144.0+ bytes


In [12]:
# create the fact table  
fact_table =  df.merge(passenger_count_dim, on='passenger_count') \
                .merge(trip_distance_dim, on='trip_distance') \
                .merge(rate_code_dim, on='RatecodeID') \
                .merge(pickup_location_dim, on=['pickup_longitude', 'pickup_latitude']) \
                .merge(dropoff_location_dim, on=['dropoff_longitude', 'dropoff_latitude'])\
                .merge(datetime_dim, on=['tpep_pickup_datetime','tpep_dropoff_datetime']) \
                .merge(payment_type_dim, on='payment_type') \
                [['trip_id', 'VendorID', 'datetime_id', 'passenger_count_id',
                'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
                'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
                'improvement_surcharge', 'total_amount']]

In [13]:
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,9.0,0.5,0.5,2.05,0.0,0.3,12.35
1,1496,2,1491,0,0,0,N,1481,1484,0,10.5,0.0,0.5,2.26,0.0,0.3,13.56
2,2840,2,2834,0,0,0,N,2816,2819,0,9.5,0.0,0.5,1.25,0.0,0.3,11.55
3,3495,2,3488,0,0,0,N,3465,3470,0,13.5,0.0,0.5,2.00,0.0,0.3,16.30
4,3930,2,3923,0,0,0,N,3899,3903,0,10.5,0.0,0.5,2.26,0.0,0.3,13.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,66038,1,65943,0,257,3,N,64896,65105,3,170.0,0.0,0.0,0.00,0.0,0.3,170.30
99996,81770,1,81651,0,257,3,N,80276,80547,3,10.0,0.0,0.0,0.00,0.0,0.3,10.30
99997,87280,2,87152,4,257,1,N,85670,85971,3,-20.0,-0.5,0.0,0.00,0.0,-0.3,-20.80
99998,53953,2,53874,4,1060,1,N,53081,53222,3,-25.5,0.0,0.0,0.00,0.0,-0.3,-25.80
